# PRISM Rössler System Validation

**Purpose**: Validate PRISM against a second deterministic chaotic system.

## Rössler System Equations

$$\frac{dx}{dt} = -y - z$$
$$\frac{dy}{dt} = x + ay$$
$$\frac{dz}{dt} = b + z(x - c)$$

**Parameters**: $a = 0.2$, $b = 0.2$, $c = 5.7$

## Characteristics vs Lorenz

| Property | Lorenz | Rössler |
|----------|--------|--------|
| Attractor | Two-winged butterfly | Single folded band |
| Regime changes | Frequent lobe switches | Intermittent spikes |
| Chaos type | Stretching + folding | Stretching + folding |
| Complexity | Higher | Lower (simpler structure) |

In [ ]:
import numpy as np
import polars as pl
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from datetime import date, timedelta

import sys
sys.path.insert(0, '..')
from prism.db.parquet_store import get_parquet_path
from prism.db.polars_io import write_parquet_atomic

## 1. Generate Rössler Trajectory

In [ ]:
def rossler_system(t, state, a=0.2, b=0.2, c=5.7):
    """Rössler ODEs."""
    x, y, z = state
    return [
        -y - z,
        x + a * y,
        b + z * (x - c)
    ]

# Integration parameters
t_span = (0, 500)  # Longer time for Rössler (slower dynamics)
dt = 0.05
t_eval = np.arange(t_span[0], t_span[1], dt)
initial_state = [1.0, 1.0, 1.0]

sol = solve_ivp(
    rossler_system,
    t_span,
    initial_state,
    t_eval=t_eval,
    method='RK45'
)

print(f"Generated {len(sol.t):,} points")
print(f"x range: [{sol.y[0].min():.2f}, {sol.y[0].max():.2f}]")
print(f"y range: [{sol.y[1].min():.2f}, {sol.y[1].max():.2f}]")
print(f"z range: [{sol.y[2].min():.2f}, {sol.y[2].max():.2f}]")

## 2. Ground Truth: Spike Detection

Rössler has intermittent z-spikes when trajectory folds back.

In [ ]:
# Detect spikes in z (regime events)
z_threshold = np.percentile(sol.y[2], 90)  # Top 10% are spikes
is_spike = sol.y[2] > z_threshold

# Count spike events (transitions into spike state)
spike_starts = np.diff(is_spike.astype(int))
n_spikes = (spike_starts == 1).sum()

# Inter-spike intervals
spike_indices = np.where(spike_starts == 1)[0]
inter_spike_intervals = np.diff(spike_indices) * dt

print(f"Z-spike threshold: {z_threshold:.2f}")
print(f"Number of spikes: {n_spikes}")
print(f"Mean inter-spike interval: {inter_spike_intervals.mean():.2f} time units")
print(f"Std inter-spike interval: {inter_spike_intervals.std():.2f} time units")

In [ ]:
# Visualize attractor
fig = plt.figure(figsize=(14, 5))

# 3D attractor
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(sol.y[0], sol.y[1], sol.y[2], lw=0.3, alpha=0.7)
ax1.set_xlabel('x'); ax1.set_ylabel('y'); ax1.set_zlabel('z')
ax1.set_title('Rössler Attractor')

# x-y projection (the famous spiral)
ax2 = fig.add_subplot(132)
ax2.plot(sol.y[0], sol.y[1], lw=0.3, alpha=0.7)
ax2.set_xlabel('x'); ax2.set_ylabel('y')
ax2.set_title('x-y Projection (spiral)')

# Signal with spikes
ax3 = fig.add_subplot(133)
t_plot = sol.t[:2000]
z_plot = sol.y[2][:2000]
ax3.plot(t_plot, z_plot, lw=0.5)
ax3.axhline(y=z_threshold, color='red', linestyle='--', alpha=0.5, label='spike threshold')
ax3.set_xlabel('Time'); ax3.set_ylabel('z')
ax3.set_title('z(t) showing intermittent spikes')
ax3.legend()

plt.tight_layout()
plt.show()

## 3. Convert to PRISM Observations

In [ ]:
base_date = date(2020, 1, 1)
n_points = len(sol.t)

rows = []
for i in range(n_points):
    obs_date = base_date + timedelta(days=i)
    rows.append({'signal_id': 'rossler_x', 'obs_date': obs_date, 'value': sol.y[0][i]})
    rows.append({'signal_id': 'rossler_y', 'obs_date': obs_date, 'value': sol.y[1][i]})
    rows.append({'signal_id': 'rossler_z', 'obs_date': obs_date, 'value': sol.y[2][i]})
    rows.append({'signal_id': 'rossler_spike', 'obs_date': obs_date, 'value': float(is_spike[i])})

obs_df = pl.DataFrame(rows)
print(f"Created {len(obs_df):,} observations")
obs_df.head(8)

In [ ]:
# Save to PRISM
default_obs_path = get_parquet_path('raw', 'observations')

if default_obs_path.exists():
    existing = pl.read_parquet(default_obs_path)
    existing = existing.filter(~pl.col('signal_id').str.starts_with('rossler_'))
    combined = pl.concat([existing, obs_df])
    write_parquet_atomic(combined, default_obs_path)
    print(f"Appended to {default_obs_path}")
else:
    write_parquet_atomic(obs_df, default_obs_path)
    print(f"Created {default_obs_path}")

## 4. Run PRISM

In [ ]:
from prism.entry_points.signal_vector import run_sliding_vectors

result = run_sliding_vectors(
    signals=['rossler_x', 'rossler_y', 'rossler_z', 'rossler_spike'],
    verbose=True
)
print(f"\nProcessed: {result}")

## 5. Analyze Results

In [ ]:
vec = pl.read_parquet(get_parquet_path('vector', 'signal'))
vec = vec.filter(pl.col('signal_id').str.starts_with('rossler_'))

print(f"Total rows: {len(vec):,}")
print(f"Engines: {vec['engine'].unique().to_list()}")

In [ ]:
# Compare x variable metrics
x_metrics = vec.filter(pl.col('signal_id') == 'rossler_x')

key_metrics = [
    ('hurst', 'hurst_exponent'),
    ('entropy', 'sample_entropy'),
    ('entropy', 'permutation_entropy'),
    ('rqa', 'determinism'),
    ('rqa', 'entropy'),
    ('rqa', 'laminarity'),
]

print("rossler_x Key Metrics:")
print("=" * 50)
for engine, metric in key_metrics:
    vals = x_metrics.filter(
        (pl.col('engine') == engine) & 
        (pl.col('metric_name') == metric)
    )['metric_value']
    if len(vals) > 0:
        print(f"{metric:<25} {vals.mean():>10.4f} +/- {vals.std():>8.4f}")

## 6. Lorenz vs Rössler Comparison

In [ ]:
# Load both systems
vec_all = pl.read_parquet(get_parquet_path('vector', 'signal'))

def get_system_metrics(vec_df, prefix):
    x_df = vec_df.filter(pl.col('signal_id') == f'{prefix}_x')
    metrics = {}
    for eng, met in [('rqa', 'determinism'), ('rqa', 'entropy'), ('entropy', 'sample_entropy'), ('hurst', 'hurst_exponent')]:
        vals = x_df.filter((pl.col('engine') == eng) & (pl.col('metric_name') == met))['metric_value']
        metrics[met] = vals.mean() if len(vals) > 0 else None
    return metrics

lorenz = get_system_metrics(vec_all, 'lorenz')
rossler = get_system_metrics(vec_all, 'rossler')

print("Comparison: Lorenz vs Rössler (x variable)")
print("=" * 55)
print(f"{'Metric':<25} {'Lorenz':>12} {'Rössler':>12}")
print("-" * 55)
for metric in ['determinism', 'entropy', 'sample_entropy', 'hurst_exponent']:
    l_val = f"{lorenz.get(metric, 0):.4f}" if lorenz.get(metric) else "N/A"
    r_val = f"{rossler.get(metric, 0):.4f}" if rossler.get(metric) else "N/A"
    print(f"{metric:<25} {l_val:>12} {r_val:>12}")

## 7. Validation Summary

In [ ]:
def get_metric_mean(df, engine, metric):
    vals = df.filter(
        (pl.col('engine') == engine) & 
        (pl.col('metric_name') == metric)
    )['metric_value']
    return vals.mean() if len(vals) > 0 else None

checks = [
    ('RQA Determinism > 0.9', get_metric_mean(x_metrics, 'rqa', 'determinism'), lambda x: x > 0.9 if x else False),
    ('RQA Laminarity > 0.8', get_metric_mean(x_metrics, 'rqa', 'laminarity'), lambda x: x > 0.8 if x else False),
    ('Sample Entropy < 0.5', get_metric_mean(x_metrics, 'entropy', 'sample_entropy'), lambda x: x < 0.5 if x else False),
]

print("PRISM Rössler Validation Results")
print("=" * 60)
for name, value, check_fn in checks:
    passed = check_fn(value)
    status = 'PASS' if passed else 'FAIL'
    val_str = f"{value:.4f}" if value else "N/A"
    print(f"{name:<40} {val_str:>10} {status:>6}")

print(f"\nGround Truth: {n_spikes} z-spikes detected")